# Практика
Используемые библиотеки

In [1]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing


## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [3]:
#headers={
#    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
#}
#def get_content(url):
#    with requests.Session() as req:
#        req.headers.update(headers)
#        r = req.get(url).content
#    return r

#url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
#s = get_content(url)
#df=pd.read_csv(io.StringIO(s.decode('UTF8')))


### Если не работает загрузка on-line
df = pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';')
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [9]:
df = pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';',index_col=0)
df

,Единица измерений,Результат анализа,Норматив
Показатель,,,
pH,единицы pH,8.4,в пределах 6-9
Запах,баллы,1,не более 2-3
Цветность,градусы,б/цвета,не более 30
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
Нитраты (по NO3),мг/дм3,24,не более 45
Фосфаты (P),мг/дм3,0.36,"не более 3,5"
Хлориды (Cl),мг/дм3,200,не более 350


In [10]:
# Ваше решение
#Создаем колонку "Индикатор": приводим к единому шаблону столбец "Норматив" ("в пределах" и "не более")
df['Индикатор'] = df['Норматив'].str.replace(r"в пределах (\d+)-(\d+)",r"\1:\2",regex=True).str.replace(r"не более.+?(\d+,?\d*)$", r"0:\1",regex=True)

#В колонке "Результат анализа" заменяем значение "б/цвета" на 0
df['Результат анализа'] = df['Результат анализа'].replace('б/цвета','0')

#Создаем колонки "нижняя граница" и "верхня граница"
df[['lower','upper']] = df['Индикатор'].str.replace(',','.').str.extract(r'(.*):(.*)')

#Создаем колонку "Результат" и записываем  true в случае если результат входит в норму
df["Результат"] = df['Результат анализа'].astype(float).between(df.lower.astype(float),df.upper.astype(float))

df



,Единица измерений,Результат анализа,Норматив,Индикатор,lower,upper,Результат
Показатель,,,,,,,
pH,единицы pH,8.4,в пределах 6-9,6:9,6,9,True
Запах,баллы,1,не более 2-3,0:3,0,3,True
Цветность,градусы,0,не более 30,0:30,0,30,True
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,7:10,7,10,True
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5","0:1,5",0,1.5,True
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3","0:3,3",0,3.3,True
Нитраты (по NO3),мг/дм3,24,не более 45,0:45,0,45,True
Фосфаты (P),мг/дм3,0.36,"не более 3,5","0:3,5",0,3.5,True
Хлориды (Cl),мг/дм3,200,не более 350,0:350,0,350,True


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [12]:
# Ваше решение
import random


In [22]:
# Заполняем ящик
B = ["orange"]*5 + ['apple']*4

# Задаем число экспериментов
n = 100000

# Количество выборки: достать апельсин 3 раза
a = 3


A = 0
for _ in range(n):
    random.shuffle(B)
    result = all(x == 'orange' for x in B[:a])
    A += result
print(A/n)


0.11906


### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [ ]:
# Ваше решение

In [29]:
#Если мастер должен проверить всего две детали, то в первом случае он должен будет вытащить не стандартную деталь, а втором стандартную
#Кейс первый: мастер вытаскивает не стандартную деталь. Остается 7 стандартных и 2 не стандартных
B1 = ["standart"]*7 + ['noStandart']*2

#Кейс второй: мастер вытаскивает стандартную деталь. Остается 6 стандартных и 2 не стандартных
B2 = ["standart"]*6 + ['noStandart']*2

# задаем число экспериментов
n = 100000

# формируем выборку
A = pd.DataFrame({"A":list(map(lambda a: random.choice([
random.choice(B1),
random.choice(B2)
]), range(n)

))

})

# считаем статистику выпадений черных шаров
P_noStandart = len(A[A.A=='noStandart'])/A.shape[0]
P_noStandart

0.23495